# Test the Table Commands

In [14]:
%load_ext autoreload
%autoreload 2
import sys, os
scriptpath = '..'
%load_ext sql
sys.path.append(os.path.abspath(scriptpath))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [28]:
from honeypot_redshift import honeypot_redshift

In [29]:
hprs = honeypot_redshift(config_file='../aws.cfg')

In [21]:
hprs.attach_iam_role()

Creating a new IAM Role
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name honeypotRole already exists.
Attaching IAM policy


In [35]:
hprs.db_connect()

In [24]:
hprs.delete_tables()

Dropping table: staging_honeypot
Dropping table: staging_ipgeo
Dropping table: staging_reputation
Dropping table: dim_glastopf
Dropping table: dim_amun
Dropping table: dim_dionaea
Dropping table: dim_snort
Dropping table: dim_ipgeo
Dropping table: dim_reputation
Dropping table: fact_attacks


In [25]:
hprs.create_tables()

Creating table: staging_honeypot
Creating table: staging_ipgeo
Creating table: staging_reputation
Creating table: dim_glastopf
Creating table: dim_amun
Creating table: dim_dionaea
Creating table: dim_snort
Creating table: dim_ipgeo
Creating table: dim_reputation
Creating table: fact_attacks


In [26]:
hprs.copy_into_tables()

Copying into table: staging_honeypot


InternalError: User arn:aws:redshift:us-west-2:493772843958:dbuser:honeypotcluster/honeypotuser is not authorized to assume IAM Role arn:aws:iam::493772843958:role/honeypotRole
DETAIL:  
  -----------------------------------------------
  error:  User arn:aws:redshift:us-west-2:493772843958:dbuser:honeypotcluster/honeypotuser is not authorized to assume IAM Role arn:aws:iam::493772843958:role/honeypotRole
  code:      8001
  context:   IAM Role=arn:aws:iam::493772843958:role/honeypotRole
  query:     781
  location:  xen_aws_credentials_mgr.cpp:321
  process:   padbmaster [pid=23468]
  -----------------------------------------------



In [70]:
hprs.insert_into_tables()

Inserting into table: dim_glastopf
Inserting into table: dim_amun
Inserting into table: dim_dionaea
Inserting into table: dim_snort
Inserting into table: dim_ipgeo
Inserting into table: dim_reputation
Inserting into table: fact_attacks


In [31]:
hprs.IAM_ROLE

'arn:aws:iam::493772843958:role/honeypotRole'

In [34]:
hprs.get_cluster_info()

{'ClusterIdentifier': 'honeypotcluster',
 'NodeType': 'dc2.large',
 'ClusterStatus': 'available',
 'MasterUsername': 'honeypotuser',
 'DBName': 'honeypot',
 'Endpoint': {'Address': 'honeypotcluster.cd5n9t7oif0x.us-west-2.redshift.amazonaws.com',
  'Port': 5439},
 'VpcId': 'vpc-7cbe4719',
 'NumberOfNodes': 4}

Need to figure out how to pass a SQL query and get the data back.

In [36]:
cur = hprs.conn.cursor()

In [61]:
cur.execute('SELECT count(*) FROM staging_honeypot;')
#for record in cur:
#    print(record[0])
#hprs.conn.commit()

In [62]:
cur.fetchall()

[(994142,)]

In [63]:
cur.execute('SELECT count(*) from attacks;')
cur.fetchall()

[(993990,)]

In [65]:
cur.execute('SELECT count(*) from dionaea_events')
cur.fetchall()

[(575180,)]

In [72]:
cur.execute('SELECT attacker_region, attacker_country, count(*) as count from attacks\
            GROUP BY attacker_region, attacker_country\
            ORDER BY count DESC\
            LIMIT 10;')
cur.fetchall()

[('', 'France', 85865),
 ('Jiangsu', 'China', 66066),
 ('Zhejiang', 'China', 60834),
 ('Arizona', 'United States', 55977),
 ('', 'Russia', 42869),
 ('', 'United Kingdom', 35344),
 ('', '', 30336),
 ('Missouri', 'United States', 22731),
 ('', 'Singapore', 22670),
 ('', 'United States', 22270)]

In [70]:
hprs.conn.commit()

Project steps:

X* Create schema for dimension tables
X* Build Insert statements for dimension tables
X* Create schema for fact table
X * Build Insert statements for fact table
X* Save schema image. 
X* change honeypot data to be whole set
X* Code documentation almost done...
x* Write script to setup cluster...
x  * Include 30-second "checks" on the database build...
x* Write script to tear down cluster
x * Build in data quality checks.  
x* Write script to do all other database wrangling on-the-fly... (etl) -- Need data quality checks still
x* Write data dictionary
x* Change *analysis* folder to *working*. 
x* Write Readme
* Write blog post (or other write-up). 